In [1]:
import pandas as pd

In [5]:
pwd

'C:\\Users\\yeliz.sari\\Downloads'

In [8]:
df = pd.read_excel('C:\\Users\\yeliz.sari\\Downloads\\online_retail_II.xlsx')

In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [10]:
df = df.copy()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  float64       
 7   Country      525461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 32.1+ MB


In [12]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [13]:
df.shape

(525461, 8)

In [14]:
df.describe().round()

,Quantity,Price,Customer ID
count,525461.0,525461.0,417534.0
mean,10.0,5.0,15361.0
std,107.0,146.0,1681.0
min,-9600.0,-53594.0,12346.0
25%,1.0,1.0,13983.0
50%,3.0,2.0,15311.0
75%,10.0,4.0,16799.0
max,19152.0,25111.0,18287.0


In [15]:
df = df[df['Quantity'] > 0 ]

In [16]:
df = df[df['Price'] > 0]

In [17]:
df.describe().round()

,Quantity,Price,Customer ID
count,511566.0,511566.0,407664.0
mean,11.0,4.0,15369.0
std,87.0,64.0,1680.0
min,1.0,0.0,12346.0
25%,1.0,1.0,13997.0
50%,3.0,2.0,15321.0
75%,10.0,4.0,16812.0
max,19152.0,25111.0,18287.0


In [18]:
df.isnull().sum()

Invoice             0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
Price               0
Customer ID    103902
Country             0
dtype: int64

In [19]:
df.quantile(q=[0, 0.25, 0.50, 0.75,1])

,Quantity,Price,Customer ID
0.00,1.0,0.001,12346.0
0.25,1.0,1.250,13997.0
0.50,3.0,2.100,15321.0
0.75,10.0,4.210,16812.0
1.00,19152.0,25111.090,18287.0


In [20]:
df['TotalPrice'] = df['Quantity'] * df['Price']

In [22]:
df = df[~df['Invoice'].str.contains('C', na=False)]

In [23]:
df['InvoiceDate'].max()

Timestamp('2010-12-09 20:01:00')

In [24]:
import datetime as dt
today_date = dt.datetime(2011,12,10)

In [25]:
today_date

datetime.datetime(2011, 12, 10, 0, 0)

In [26]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                    'Invoice': lambda num: num.nunique(),
                                    'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [27]:
rfm.columns = ['recency', 'frequency','monetary']

In [28]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.0,455.172542,96.861457,365.00,382.0000,417.00,500.0000,738.00
frequency,4312.0,4.455705,8.170213,1.00,1.0000,2.00,5.0000,205.00
monetary,4312.0,2048.238236,8914.481280,2.95,307.9875,706.02,1723.1425,349164.35


In [29]:
# recency: müşteriler ortalama 200 gün önce alışveriş yapmışlar
# frequency: ortalama frekans 6, 75% için ortalama 7 günlük bir sıklık var ama maksimum'a baktığımızda muhtemelen Pareto diyebiliriz.

In [30]:
rfm['recency_score']  = pd.qcut(rfm['recency'], 5, [5, 4, 3, 2, 1])
rfm['frequency_score']  = pd.qcut(rfm['frequency'].rank(method="first"), 5, [1, 2, 3, 4, 5])
rfm['monetary_score']  = pd.qcut(rfm['monetary'], 5, [1, 2, 3, 4, 5])

In [31]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.0,529,11,372.86,2,5,2
12347.0,367,2,1323.32,5,2,4
12348.0,438,1,222.16,2,1,1
12349.0,407,3,2671.14,3,3,5
12351.0,375,1,300.93,5,1,2


In [32]:
rfm['RFM_SCORE'] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [33]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.0,529,11,372.86,2,5,2,25
12347.0,367,2,1323.32,5,2,4,52
12348.0,438,1,222.16,2,1,1,21
12349.0,407,3,2671.14,3,3,5,33
12351.0,375,1,300.93,5,1,2,51


In [34]:
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [35]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0,529,11,372.86,2,5,2,25,cant_loose
12347.0,367,2,1323.32,5,2,4,52,potential_loyalists
12348.0,438,1,222.16,2,1,1,21,hibernating
12349.0,407,3,2671.14,3,3,5,33,need_attention
12351.0,375,1,300.93,5,1,2,51,new_customers


In [36]:
rfm[["segment","recency","frequency","monetary"]].groupby("segment").agg(["mean", "count" ])

recency        frequency           monetary      
                           mean count       mean count         mean count
segment                                                                  
about_to_sleep       417.819242   343   1.201166   343   441.320000   343
at_Risk              516.158756   611   3.073650   611  1188.878316   611
cant_loose           488.116883    77   9.116883    77  4099.450000    77
champions            371.119155   663  12.553544   663  6852.264167   663
hibernating          577.885714  1015   1.126108  1015   403.977836  1015
loyal_customers      400.287062   742   6.830189   742  2746.067353   742
need_attention       417.265700   207   2.449275   207  1060.357005   207
new_customers        372.580000    50   1.000000    50   386.199200    50
potential_loyalists  382.793037   517   2.017408   517   729.510986   517
promising            389.747126    87   1.000000    87   367.086782    87